- Since this data have been scraped from the web I am assuming there will be some manual cleaning necessary before I can perform exploratory data analysis on the dataset. Below are the steps that I took in doing so:

In [81]:
import numpy as np
import pandas as pd

# Loading the Data:

In [23]:
beers = pd.read_csv('untapped_beer_data_eng_language.csv')

In [24]:
beers.head()

,name,brewery,beer_style,abv,date_added,not_available,num_ratings,rating,beer_desc
0,Exalted,Thornbridge Brewery,Altbier,4.5% ABV,06/10/2016,NaN,542,3.70,Collaboration with Magic Rock
1,Leaf,Orbit Beers London,Altbier,6.2% ABV,12/03/2014,NaN,783,3.67,Beechwood smoked malt meets a double altbier.\...
2,Ctrl Alt Del,Wylam,Altbier,4.9% ABV,02/07/2016,NaN,318,3.65,12.4° Plato | OG 1050 | ABV 4.9%\n\nWylam & Le...
3,Hades,Ghost Brew Co,Altbier,4.5% ABV,07/16/16,NaN,228,3.62,Dry Hopped Altbier.
4,The Altbier,Anspach & Hobday,Altbier,5% ABV,10/02/2015,NaN,288,3.57,NaN


- Looking at the size of the dataframe.
- This checks I have enough data to analyse and also is a good check post-cleaning to see how many datapoints cleaning may end up removing.

In [25]:
beers.shape

(22984, 9)

# Looking at the Variables:

## Number of unique attributes per variable:

In [26]:
beers.nunique(axis=0)

name             21511
brewery           3892
beer_style         213
abv                305
date_added        3486
not_available        1
num_ratings       4666
rating             235
beer_desc        19332
dtype: int64

- As we can see, because of the categorical variables this dataset exhibits high cardinality.

## Description of the numerical columns:

In [27]:
beers.describe()

,not_available,rating
count,4357.0,22984.000000
mean,1.0,3.779146
std,0.0,0.260250
min,1.0,1.790000
25%,1.0,3.620000
50%,1.0,3.750000
75%,1.0,3.910000
max,1.0,4.740000


- Right now there are only two numerical variables in the dataframe, although I'd expect four (not available, rating, ABV and number of ratings). 
- ABV and number of ratings are being recognised as a non-numeric type, I will rectifty this below.

## A look at the null values included in the dataframe:

In [28]:
beers.isnull().sum()

name                 0
brewery              0
beer_style           0
abv                  0
date_added           0
not_available    18627
num_ratings          0
rating               0
beer_desc         3481
dtype: int64

- Not available indicates whether a beer has been discontinued (i.e. is not available for purchase).

- At present a 1.0 indicates 'not available' and a NaN indicates it is still purchasable.

- I will fill the NaN in this column with zero's and then change the column so it more easily understood.

In [29]:
# Replacing NaNs with zero's:
beers['not_available'] = beers['not_available'].fillna(0)

In [30]:
beers.not_available.value_counts()

0.0    18627
1.0     4357
Name: not_available, dtype: int64

In [31]:
# Swapping 1 for 0 in the not_available column
beers.not_available = beers.not_available.map( {0:1 , 1:0} )

# Changing the column name to be called available - so 1's now equate to a beer that is purchasable.
beers = beers.rename(columns={'not_available': 'available'})

In [32]:
beers.available.value_counts()

1    18627
0     4357
Name: available, dtype: int64

### Filling the beer_desc null values with zero for now:

- These are beers that either don't have descriptions on the can or the descriptions have not been uploaded to untapped, during EDA I will inspect what % of my dataset has no description and decide what to do about that.

In [33]:
beers['beer_desc'] = beers['beer_desc'].fillna(0)

In [34]:
beers.isnull().sum()

name           0
brewery        0
beer_style     0
abv            0
date_added     0
available      0
num_ratings    0
rating         0
beer_desc      0
dtype: int64

## Looking at the datatypes of the columns:

In [35]:
beers.dtypes

name            object
brewery         object
beer_style      object
abv             object
date_added      object
available        int64
num_ratings     object
rating         float64
beer_desc       object
dtype: object

### Inspecting why num_ratings is an object datatype:

- num_ratings is the number of ratings that the beer has received, so it should be an int type.

In [36]:
# Looking at instances where num_ratings is not numeric:
beers[beers['num_ratings'].str.isnumeric() == False]

,name,brewery,beer_style,abv,date_added,available,num_ratings,rating,beer_desc
8,Neu,Orbit Beers London,Altbier,4.7% ABV,08/26/14,0,"1,780",3.47,Pours deep amber colour with beautiful reddish...
10,London Alt,Mondo Brewing Company,Altbier,4.8% ABV,02/24/15,1,"1,227",3.45,The London Alt Beer is a original Dusseldorf-s...
19,DoppelSticky,Lagunitas Brewing Company,Altbier,7.7% ABV,09/10/2015,1,"12,203",3.87,"This fruity, estery double altbier is brewed i..."
30,Altbitter,Widmer Brothers Brewing,Altbier,9.5% ABV,09/08/2014,1,"2,276",3.78,"AltBitter, a collaboration beer between Deschu..."
39,Sticke Alt,Lion Bridge Brewing Company,Altbier,6% ABV,04/15/14,1,"1,054",3.75,A traditional German Ale that has a pleasing b...
...,...,...,...,...,...,...,...,...,...
22970,Holly King,Green Man Brewery,Winter Warmer,11% ABV,12/03/2013,1,"2,302",3.81,This rich holiday ale is perfect to warm away ...
22972,High Camp Winter IPA,Bale Breaker Brewing Company,Winter Warmer,7.3% ABV,11/22/13,1,"2,524",3.81,Named for the ski lodge atop our local White P...
22973,Marley's Lament,Alewerks Brewing Company,Winter Warmer,8% ABV,11/30/13,1,"2,137",3.80,This winter warmer is perfect for a cold day. ...
22980,Zoigl-Pils,Zoiglhaus Brewing Company,Zoigl,4.8% ABV,03/16/17,1,"2,188",3.74,2017 Great American Beer Festival Gold Medal i...


- Having looked at the csv file, there are commas, spaces and the word 'Ratings' preventing the num_ratings column being typed as an integer. Below I have removed these values from this column:

In [37]:
beers.num_ratings = [int(x.strip(' ').replace(',', '').replace('Ratings', '')) for x in beers.num_ratings]

In [38]:
# num_ratings now changed to an int column:
beers.dtypes

name            object
brewery         object
beer_style      object
abv             object
date_added      object
available        int64
num_ratings      int64
rating         float64
beer_desc       object
dtype: object

### Removing % ABV from abv column to convert it to float:

In [41]:
# Removing the spaces, % signs and ABV from column:
beers.abv = [(x.strip(' ').replace('%', '').replace('ABV', '')) for x in beers.abv]

In [42]:
beers.head()

,name,brewery,beer_style,abv,date_added,available,num_ratings,rating,beer_desc
0,Exalted,Thornbridge Brewery,Altbier,4.5,06/10/2016,1,542,3.70,Collaboration with Magic Rock
1,Leaf,Orbit Beers London,Altbier,6.2,12/03/2014,1,783,3.67,Beechwood smoked malt meets a double altbier.\...
2,Ctrl Alt Del,Wylam,Altbier,4.9,02/07/2016,1,318,3.65,12.4° Plato | OG 1050 | ABV 4.9%\n\nWylam & Le...
3,Hades,Ghost Brew Co,Altbier,4.5,07/16/16,1,228,3.62,Dry Hopped Altbier.
4,The Altbier,Anspach & Hobday,Altbier,5,10/02/2015,1,288,3.57,0


- There are still some columns with N/A in the abv:

In [59]:
beers[27:28]

,name,brewery,beer_style,abv,date_added,available,num_ratings,rating,beer_desc
27,Bourbon Barrel Aged Copper,The Olde Mecklenburg Brewery,Altbier,N/A,09/11/2016,1,173,3.79,0


In [63]:
beers[beers['abv']=='N/A ']

,name,brewery,beer_style,abv,date_added,available,num_ratings,rating,beer_desc
27,Bourbon Barrel Aged Copper,The Olde Mecklenburg Brewery,Altbier,N/A,09/11/2016,1,173,3.79,0
42,Bourbon Barrel Aged Anniversary Alt,The Olde Mecklenburg Brewery,Altbier,N/A,10/04/2014,1,163,3.74,0
45,Anniversary Alt,The Olde Mecklenburg Brewery,Altbier,N/A,03/08/2013,1,1047,3.74,Anniversary Alt is brewed every March for our ...
111,Civil Disobedience #14,Hill Farmstead Brewery,American Wild Ale,N/A,07/09/2015,0,1402,4.44,A blend of various beers/barrels from Summer 2...
115,Elaborative Five,Hill Farmstead Brewery,American Wild Ale,N/A,08/09/2016,0,2090,4.42,100% Spontaneously Fermented/Coolshipped beer ...
...,...,...,...,...,...,...,...,...,...
22277,Uberon,Bell's Brewery,Wheat Beer - American Pale Wheat,N/A,06/16/13,1,7015,3.93,Barrel-aging this beer coaxes flavors out of T...
22393,Peach Crumble,Lil Beaver Brewery,Wheat Beer - Other,N/A,03/03/2020,1,637,3.86,"Wheat Ale with Peach, Cinnamon, and Vanill"
22427,Candy Floss Pomegranate Wheat,Heavy Riff Brewing Company,Wheat Beer - Witbier,N/A,09/24/16,1,302,3.77,This pomegranate wheat is one of our originals...
22656,Naked for Satan Ale,Matilda Bay Brewing Company,Wheat Beer - Witbier,N/A,11/16/11,1,258,3.51,0


- Since these are only 202 n/a values out of a dataframe of 20,000 values I am happy to drop N/A abv rows as I think abv is an important feature, and filling with either zero's, mean, median or knn would not be very meaningful.

In [66]:
# Removing further spaces and changing N/A string to 0 string to convert to numeric later
beers.abv = [(x.strip(' ').replace('N/A', '0')) for x in beers.abv]

In [69]:
beers[27:28]

,name,brewery,beer_style,abv,date_added,available,num_ratings,rating,beer_desc
27,Bourbon Barrel Aged Copper,The Olde Mecklenburg Brewery,Altbier,0,09/11/2016,1,173,3.79,0


In [78]:
beers.head()

,name,brewery,beer_style,abv,date_added,available,num_ratings,rating,beer_desc
0,Exalted,Thornbridge Brewery,Altbier,4.5,2016-06-10,1,542,3.70,Collaboration with Magic Rock
1,Leaf,Orbit Beers London,Altbier,6.2,2014-12-03,1,783,3.67,Beechwood smoked malt meets a double altbier.\...
2,Ctrl Alt Del,Wylam,Altbier,4.9,2016-02-07,1,318,3.65,12.4° Plato | OG 1050 | ABV 4.9%\n\nWylam & Le...
3,Hades,Ghost Brew Co,Altbier,4.5,2016-07-16,1,228,3.62,Dry Hopped Altbier.
4,The Altbier,Anspach & Hobday,Altbier,5.0,2015-10-02,1,288,3.57,0


In [71]:
# Changing column to a numeric column:
beers['abv'] = pd.to_numeric(beers['abv'])

In [74]:
beers.shape

(22984, 9)

In [75]:
# Dropping any rows in abv that are equal to zero:
beers = beers[beers['abv'] != 0]

In [76]:
beers.shape  #202 rows removed

(22782, 9)

In [73]:
# abv now changed to an float column:
beers.dtypes

name            object
brewery         object
beer_style      object
abv            float64
date_added      object
available        int64
num_ratings      int64
rating         float64
beer_desc       object
dtype: object

### Converting date_added column to datetime:

In [77]:
beers['date_added'] =  pd.to_datetime(beers['date_added'])

In [79]:
beers.dtypes

name                   object
brewery                object
beer_style             object
abv                   float64
date_added     datetime64[ns]
available               int64
num_ratings             int64
rating                float64
beer_desc              object
dtype: object

# Creating a new column for Amalgamated Beer Styles:

- There are 213 (see below) unique beer styles included in the dataset, with some being very similar to others (e.g. 'Lager - Amber', 'Lager - American', 'Lager - American Amber / Red', 'Lager - American Light').
- I will create a new column where all IPA's, all lagers, all pale ales etc are grouped together.

In [93]:
beers.beer_style.nunique()

213

In [94]:
# Creating a function to match words in the beer_style column to those I've listed as popular beers:
# Popular beers are those that have more than one variant - i.e. Bock Doppelbok and Bock - Eisbock or Brown Ale - English and 
# Brown Ale - American

def extract_beer_supgroup(beer):
    pop_beer =  ['Barleywine', 'Belgian', 'Bock', 'Brown Ale', 'Cider', 'Farmhouse Ale', 'IPA', 'Lager', 
                'Lambic', 'Mead', 'Pale Ale', 'Pilsner', 'Porter', 'Red Ale', 'Sour', 'Stout', 'Strong Ale', 'Wheat Beer']
    beer_supgroup = beer.split('-')[0].strip(' ')
    if beer_supgroup in pop_beer:
        supgroup = beer_supgroup
    elif beer.split(' ')[0] == 'Belgian':
        supgroup = 'Belgian'
    else:
        supgroup = 'Other'
        
    return supgroup   

In [95]:
beers['beer_style_group'] = [extract_beer_supgroup(x) for x in beers.beer_style]

In [97]:
# Checking that my function has executed correctly:
pd.set_option('display.max_rows', 230)
beers[['beer_style', 'beer_style_group']].drop_duplicates()

,beer_style,beer_style_group
0,Altbier,Other
74,American Wild Ale,Other
152,Australian Sparkling Ale,Other
155,Golden Ale,Other
191,Barleywine - American,Barleywine
298,Barleywine - English,Barleywine
425,Barleywine - Other,Barleywine
501,Belgian Blonde,Belgian
601,Belgian Dubbel,Belgian
704,Belgian Quadrupel,Belgian


In [98]:
beers['beer_style_group'].value_counts()

Other            5596
IPA              3663
Stout            2506
Lager            1615
Pale Ale         1441
Sour             1292
Porter            990
Cider             841
Red Ale           622
Brown Ale         593
Belgian           587
Pilsner           543
Farmhouse Ale     515
Wheat Beer        485
Mead              358
Bock              338
Barleywine        310
Strong Ale        245
Lambic            242
Name: beer_style_group, dtype: int64

# Saving resulting Dataframe to csv file:

In [99]:
beers.to_csv('beers_data_cleaned.csv')